In [1]:
import pandas as pd
pd.options.display.max_rows = 10
import numpy as np 
# pd.set_option('display.max_colwidth', None)
import os
import torch

# Loading Model Directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [2]:
# now zipped in Summarization_and_preprocessig_dfs.zip and Summarization_and_preprocessig_dfs_2.zip
df = pd.read_csv('s_and_h_preprocess_3_no_spell.csv', encoding='ISO-8859-1')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
df

,True_Index,File_Index,episode_number,title,download_url,publication_date,transcription_file,transcript,generated_transcripts,cleaned_generated_transcripts
0,0,39,1919,Ep. 1919 - Nikki Haley Is OUT,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-06 16:20:13+00:00,BEN/Ep. 1919 - Nikki Haley Is OUT-transcript.txt,well end road nikki haley former un ambassador...,WELL IT'S THE END OF THE ROAD FOR NICKEY HALEY...,well it is the end of the road for nickey hale...
1,1,40,1918,Ep. 1918 - Peeing in Your Own Eyes To Stop Don...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-05 16:03:39+00:00,BEN/Ep. 1918 - Peeing in Your Own Eyes To Stop...,well yesterday supreme court disappointed ever...,WELL YESTERDAY THE SUPREME COURT DISAPPOINTED ...,well yesterday the supreme court disappointed ...
2,2,41,1917,Ep. 1917 - UNANIMOUS Supreme Court Puts Trump ...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-04 16:14:37+00:00,BEN/Ep. 1917 - UNANIMOUS Supreme Court Puts Tr...,carshield 1s love get open road see country fe...,ALREADY FOLK SO BIG WIND FOR A DONAL TRUMP AFT...,already folk big wind for a donal trump aft th...
3,3,42,1916,Ep. 1916 - Duel At The Border,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-01 16:18:45+00:00,BEN/Ep. 1916 - Duel At The Border-transcript.txt,0 8 million border crossings war middle east u...,A FOLKS EARLIER THIS WEEK YOU REMEMBER THAT WE...,a folks earlier this week you remember that we...
4,4,43,1915,Ep. 1915 - THE DIRTY SECRET: Trump Is The 2024...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-02-29 16:15:29+00:00,BEN/Ep. 1915 - THE DIRTY SECRET_ Trump Is The ...,angel envy bourbons 0s angels envy envy good t...,SO BEFORE WE GET TO THE BIG NEWS OF THE DAY I ...,before we get to the big news of the day i wan...
...,...,...,...,...,...,...,...,...,...,...
498,498,539,4,"How to Defeat Jetlag, Shift Work & Sleeplessness",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-25 09:30:00+00:00,"HUBERMAN/How to Defeat Jetlag, Shift Work & Sl...",welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN LAPONCAS O SCIENCE S...,welcome to the huberman laponcas o science sci...
499,499,540,3,"Using Science to Optimize Sleep, Learning & Me...",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-18 09:00:00+00:00,"HUBERMAN/Using Science to Optimize Sleep, Lear...",welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LAPONCAS T A SCIENCE ...,welcome to the huberman laponcas t a science s...
500,500,541,2,Be More Alert When Awake,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-11 13:05:00+00:00,HUBERMAN/Be More Alert When Awake-transcript.txt,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMEN LAPONCAS AT SCIENCE S...,welcome to the hubermen laponcas at science sc...
501,501,542,1,How Your Brain Works & Changes,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-04 16:13:00+00:00,HUBERMAN/How Your Brain Works & Changes-transc...,welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN TAONCA T AS SCIENCE ...,welcome to the huberman taonca t as science sc...


In [2]:
device = torch.device("mps")  # Use MPS device if available
model.to(device) 

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [5]:
# Function to summarize text
def summarize_text(text, device): 
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs, max_length=300, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    # summary_ids = model.generate(inputs, max_length=1000, min_length=350, length_penalty=2.0, num_beams=6, early_stopping=True) -> For Andre Huberman
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# Ensuring the SUMMARIES directory exists
summaries_dir = 'SUMMARIES'
if not os.path.exists(summaries_dir):
    os.makedirs(summaries_dir)

# Iterating through a subset of the DataFrame and summarize texts
start_index = 306  # define your start index here
end_index = 504  # define your end index here

for index, row in df.iloc[start_index:end_index].iterrows():
    print(f"Processing summary for df ['{row['True_Index']}'], df['{row['File_Index']}'], df['{row['title']}']")
    summary = summarize_text(row['cleaned_generated_transcripts'], device=device)
    # Writing summary to file
    summary_file_path = os.path.join(summaries_dir, f"{index}.txt")
    with open(summary_file_path, 'w') as file:
        file.write(summary)
    print(f"Processed summary for df ['{row['True_Index']}'], df['{row['File_Index']}'], df['{row['title']}]")


In [13]:
df.iloc[306:505]

,True_Index,File_Index,episode_number,title,download_url,publication_date,transcription_file,transcript,generated_transcripts,cleaned_generated_transcripts
306,306,345,198,How Placebo Effects Work to Change Our Biology...,https://www.podtrac.com/pts/redirect.mp3/pdst....,2024-03-04 09:00:00+00:00,HUBERMAN/How Placebo Effects Work to Change Ou...,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMANLAPONENS A SCIENCE SCI...,welcome to the hubermanlaponens a science scie...
307,307,346,197,"AMA #16: Sleep, Vertigo, TBI, OCD, Tips for Tr...",https://www.podtrac.com/pts/redirect.mp3/pdst....,2024-02-29 23:00:00+00:00,"HUBERMAN/AMA #16_ Sleep, Vertigo, TBI, OCD, Ti...",welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LADPODCA T CUST SCIEN...,welcome to the huberman ladpodca t cust scienc...
308,308,347,196,Dr. Becky Kennedy: Protocols for Excellent Par...,https://www.podtrac.com/pts/redirect.mp3/pdst....,2024-02-26 09:00:00+00:00,HUBERMAN/Dr. Becky Kennedy_ Protocols for Exce...,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMANLANCONTE O SCIENCE SCI...,welcome to the hubermanlanconte o science scie...
309,309,348,195,Dr. Mark D'Esposito: How to Optimize Cognitive...,https://www.podtrac.com/pts/redirect.mp3/pdst....,2024-02-19 09:00:00+00:00,HUBERMAN/Dr. Mark D'Esposito_ How to Optimize ...,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMANACONTANCE OF SCIENCE S...,welcome to the hubermanacontance of science sc...
310,310,349,194,How to Improve Oral Health & Its Critical Role...,https://www.podtrac.com/pts/redirect.mp3/pdst....,2024-02-12 09:00:00+00:00,HUBERMAN/How to Improve Oral Health & Its Crit...,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LAPOE A SCIENCE SCIEN...,welcome to the huberman lapoe a science scienc...
...,...,...,...,...,...,...,...,...,...,...
498,498,539,4,"How to Defeat Jetlag, Shift Work & Sleeplessness",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-25 09:30:00+00:00,"HUBERMAN/How to Defeat Jetlag, Shift Work & Sl...",welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN LAPONCAS O SCIENCE S...,welcome to the huberman laponcas o science sci...
499,499,540,3,"Using Science to Optimize Sleep, Learning & Me...",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-18 09:00:00+00:00,"HUBERMAN/Using Science to Optimize Sleep, Lear...",welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LAPONCAS T A SCIENCE ...,welcome to the huberman laponcas t a science s...
500,500,541,2,Be More Alert When Awake,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-11 13:05:00+00:00,HUBERMAN/Be More Alert When Awake-transcript.txt,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMEN LAPONCAS AT SCIENCE S...,welcome to the hubermen laponcas at science sc...
501,501,542,1,How Your Brain Works & Changes,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-04 16:13:00+00:00,HUBERMAN/How Your Brain Works & Changes-transc...,welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN TAONCA T AS SCIENCE ...,welcome to the huberman taonca t as science sc...


In [4]:
file_contents = []

for index in df.index:
    file_path = f'SUMMARIES/{index}.txt'

    try:
        with open (file_path, 'r') as file:
            content = file.read()
    
    except FileNotFoundError:
        content = None

    file_contents.append(content)


In [6]:
df['summaries'] = file_contents

In [7]:
df

,True_Index,File_Index,episode_number,title,download_url,publication_date,transcription_file,transcript,generated_transcripts,cleaned_generated_transcripts,summaries
0,0,39,1919,Ep. 1919 - Nikki Haley Is OUT,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-06 16:20:13+00:00,BEN/Ep. 1919 - Nikki Haley Is OUT-transcript.txt,well end road nikki haley former un ambassador...,WELL IT'S THE END OF THE ROAD FOR NICKEY HALEY...,well it is the end of the road for nickey hale...,well it is the end of the road for nickey hale...
1,1,40,1918,Ep. 1918 - Peeing in Your Own Eyes To Stop Don...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-05 16:03:39+00:00,BEN/Ep. 1918 - Peeing in Your Own Eyes To Stop...,well yesterday supreme court disappointed ever...,WELL YESTERDAY THE SUPREME COURT DISAPPOINTED ...,well yesterday the supreme court disappointed ...,well yesterday the supreme court disappointed ...
2,2,41,1917,Ep. 1917 - UNANIMOUS Supreme Court Puts Trump ...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-04 16:14:37+00:00,BEN/Ep. 1917 - UNANIMOUS Supreme Court Puts Tr...,carshield 1s love get open road see country fe...,ALREADY FOLK SO BIG WIND FOR A DONAL TRUMP AFT...,already folk big wind for a donal trump aft th...,The supreme court rules nine to nothing unanim...
3,3,42,1916,Ep. 1916 - Duel At The Border,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-01 16:18:45+00:00,BEN/Ep. 1916 - Duel At The Border-transcript.txt,0 8 million border crossings war middle east u...,A FOLKS EARLIER THIS WEEK YOU REMEMBER THAT WE...,a folks earlier this week you remember that we...,We ask you for tips and we have received those...
4,4,43,1915,Ep. 1915 - THE DIRTY SECRET: Trump Is The 2024...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-02-29 16:15:29+00:00,BEN/Ep. 1915 - THE DIRTY SECRET_ Trump Is The ...,angel envy bourbons 0s angels envy envy good t...,SO BEFORE WE GET TO THE BIG NEWS OF THE DAY I ...,before we get to the big news of the day i wan...,The prevalence of diversity equity ant inclusi...
...,...,...,...,...,...,...,...,...,...,...,...
498,498,539,4,"How to Defeat Jetlag, Shift Work & Sleeplessness",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-25 09:30:00+00:00,"HUBERMAN/How to Defeat Jetlag, Shift Work & Sl...",welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN LAPONCAS O SCIENCE S...,welcome to the huberman laponcas o science sci...,welcome to the huberman laponcas o science sci...
499,499,540,3,"Using Science to Optimize Sleep, Learning & Me...",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-18 09:00:00+00:00,"HUBERMAN/Using Science to Optimize Sleep, Lear...",welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LAPONCAS T A SCIENCE ...,welcome to the huberman laponcas t a science s...,welcome to the huberman laponcas t a science s...
500,500,541,2,Be More Alert When Awake,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-11 13:05:00+00:00,HUBERMAN/Be More Alert When Awake-transcript.txt,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMEN LAPONCAS AT SCIENCE S...,welcome to the hubermen laponcas at science sc...,Sleep and wakefulness govern everything about ...
501,501,542,1,How Your Brain Works & Changes,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-04 16:13:00+00:00,HUBERMAN/How Your Brain Works & Changes-transc...,welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN TAONCA T AS SCIENCE ...,welcome to the huberman taonca t as science sc...,welcome to the huberman taonca t as science sc...


In [9]:
df.rename(columns = {'transcript':'cleaned_original_transcripts'}, inplace = True) 

# df.to_csv('Final_df_with_summaries.csv')

In [10]:
df

,True_Index,File_Index,episode_number,title,download_url,publication_date,transcription_file,cleaned_original_transcripts,generated_transcripts,cleaned_generated_transcripts,summaries
0,0,39,1919,Ep. 1919 - Nikki Haley Is OUT,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-06 16:20:13+00:00,BEN/Ep. 1919 - Nikki Haley Is OUT-transcript.txt,well end road nikki haley former un ambassador...,WELL IT'S THE END OF THE ROAD FOR NICKEY HALEY...,well it is the end of the road for nickey hale...,well it is the end of the road for nickey hale...
1,1,40,1918,Ep. 1918 - Peeing in Your Own Eyes To Stop Don...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-05 16:03:39+00:00,BEN/Ep. 1918 - Peeing in Your Own Eyes To Stop...,well yesterday supreme court disappointed ever...,WELL YESTERDAY THE SUPREME COURT DISAPPOINTED ...,well yesterday the supreme court disappointed ...,well yesterday the supreme court disappointed ...
2,2,41,1917,Ep. 1917 - UNANIMOUS Supreme Court Puts Trump ...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-04 16:14:37+00:00,BEN/Ep. 1917 - UNANIMOUS Supreme Court Puts Tr...,carshield 1s love get open road see country fe...,ALREADY FOLK SO BIG WIND FOR A DONAL TRUMP AFT...,already folk big wind for a donal trump aft th...,The supreme court rules nine to nothing unanim...
3,3,42,1916,Ep. 1916 - Duel At The Border,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-01 16:18:45+00:00,BEN/Ep. 1916 - Duel At The Border-transcript.txt,0 8 million border crossings war middle east u...,A FOLKS EARLIER THIS WEEK YOU REMEMBER THAT WE...,a folks earlier this week you remember that we...,We ask you for tips and we have received those...
4,4,43,1915,Ep. 1915 - THE DIRTY SECRET: Trump Is The 2024...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-02-29 16:15:29+00:00,BEN/Ep. 1915 - THE DIRTY SECRET_ Trump Is The ...,angel envy bourbons 0s angels envy envy good t...,SO BEFORE WE GET TO THE BIG NEWS OF THE DAY I ...,before we get to the big news of the day i wan...,The prevalence of diversity equity ant inclusi...
...,...,...,...,...,...,...,...,...,...,...,...
498,498,539,4,"How to Defeat Jetlag, Shift Work & Sleeplessness",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-25 09:30:00+00:00,"HUBERMAN/How to Defeat Jetlag, Shift Work & Sl...",welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN LAPONCAS O SCIENCE S...,welcome to the huberman laponcas o science sci...,welcome to the huberman laponcas o science sci...
499,499,540,3,"Using Science to Optimize Sleep, Learning & Me...",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-18 09:00:00+00:00,"HUBERMAN/Using Science to Optimize Sleep, Lear...",welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LAPONCAS T A SCIENCE ...,welcome to the huberman laponcas t a science s...,welcome to the huberman laponcas t a science s...
500,500,541,2,Be More Alert When Awake,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-11 13:05:00+00:00,HUBERMAN/Be More Alert When Awake-transcript.txt,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMEN LAPONCAS AT SCIENCE S...,welcome to the hubermen laponcas at science sc...,Sleep and wakefulness govern everything about ...
501,501,542,1,How Your Brain Works & Changes,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-04 16:13:00+00:00,HUBERMAN/How Your Brain Works & Changes-transc...,welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN TAONCA T AS SCIENCE ...,welcome to the huberman taonca t as science sc...,welcome to the huberman taonca t as science sc...


In [16]:
# now zipped in Summarization_and_preprocessig_dfs.zip and Summarization_and_preprocessig_dfs_2.zip
df_p = pd.read_csv('s_and_h_preprocess_3_no_spell.csv')
df_p

,Unnamed: 0,True_Index,File_Index,episode_number,title,download_url,publication_date,transcription_file,transcript,generated_transcripts,cleaned_generated_transcripts
0,0,0,39,1919,Ep. 1919 - Nikki Haley Is OUT,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-06 16:20:13+00:00,BEN/Ep. 1919 - Nikki Haley Is OUT-transcript.txt,well end road nikki haley former un ambassador...,WELL IT'S THE END OF THE ROAD FOR NICKEY HALEY...,well it is the end of the road for nickey hale...
1,1,1,40,1918,Ep. 1918 - Peeing in Your Own Eyes To Stop Don...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-05 16:03:39+00:00,BEN/Ep. 1918 - Peeing in Your Own Eyes To Stop...,well yesterday supreme court disappointed ever...,WELL YESTERDAY THE SUPREME COURT DISAPPOINTED ...,well yesterday the supreme court disappointed ...
2,2,2,41,1917,Ep. 1917 - UNANIMOUS Supreme Court Puts Trump ...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-04 16:14:37+00:00,BEN/Ep. 1917 - UNANIMOUS Supreme Court Puts Tr...,carshield 1s love get open road see country fe...,ALREADY FOLK SO BIG WIND FOR A DONAL TRUMP AFT...,already folk big wind for a donal trump aft th...
3,3,3,42,1916,Ep. 1916 - Duel At The Border,https://claritaspod.com/measure/arttrk.com/p/2...,2024-03-01 16:18:45+00:00,BEN/Ep. 1916 - Duel At The Border-transcript.txt,0 8 million border crossings war middle east u...,A FOLKS EARLIER THIS WEEK YOU REMEMBER THAT WE...,a folks earlier this week you remember that we...
4,4,4,43,1915,Ep. 1915 - THE DIRTY SECRET: Trump Is The 2024...,https://claritaspod.com/measure/arttrk.com/p/2...,2024-02-29 16:15:29+00:00,BEN/Ep. 1915 - THE DIRTY SECRET_ Trump Is The ...,angel envy bourbons 0s angels envy envy good t...,SO BEFORE WE GET TO THE BIG NEWS OF THE DAY I ...,before we get to the big news of the day i wan...
...,...,...,...,...,...,...,...,...,...,...,...
498,498,498,539,4,"How to Defeat Jetlag, Shift Work & Sleeplessness",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-25 09:30:00+00:00,"HUBERMAN/How to Defeat Jetlag, Shift Work & Sl...",welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN LAPONCAS O SCIENCE S...,welcome to the huberman laponcas o science sci...
499,499,499,540,3,"Using Science to Optimize Sleep, Learning & Me...",https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-18 09:00:00+00:00,"HUBERMAN/Using Science to Optimize Sleep, Lear...",welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMAN LAPONCAS T A SCIENCE ...,welcome to the huberman laponcas t a science s...
500,500,500,541,2,Be More Alert When Awake,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-11 13:05:00+00:00,HUBERMAN/Be More Alert When Awake-transcript.txt,welcome huberman lab podcast discuss science s...,WELCOME TO THE HUBERMEN LAPONCAS AT SCIENCE S...,welcome to the hubermen laponcas at science sc...
501,501,501,542,1,How Your Brain Works & Changes,https://www.podtrac.com/pts/redirect.mp3/pdst....,2021-01-04 16:13:00+00:00,HUBERMAN/How Your Brain Works & Changes-transc...,welcome huberman lab podcast discuss science t...,WELCOME TO THE HUBERMAN TAONCA T AS SCIENCE ...,welcome to the huberman taonca t as science sc...


# **Sample Checks**

In [21]:
sample_file = 'PREPROCESSING_3/0.txt'

with open(sample_file, 'r') as file:
    sample = file.read()

summary = summarize_text(sample, device=device)

In [22]:
summary

'well end road nikki haley former un ambassador donald trump former governor south carolina dropping race today donald Trump dominated super tuesday course shock question staying reality every state except vermont trump convincingly home state south car Carolina beat 20 points beat 60 40 north carolina beat 73 23 virginia beat 63 34 particularly virginia virginia moderate republican state left wing republican state right wing republicanState beat soundly state california beat 78 18 suggests really shot nomination iowa one shot nomination Iowa pointed rhonda desantis dropped early also niki haley stayed hope win new hampshire overperformed little bit stayed past mainly think put flag rest republican party trump lose november.'

In [12]:
with open('summarization_sample_with_final_base_preprocessing.txt', 'w') as out_file: 
    out_file.write(summary)

In [23]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score('well it is the end of the road for nickey haley the former you an ambassador under a donald trump former governor of south carolina she is droping out of the race to day because donald trum dominated supertuesday. She is not going to announce an indorsement on wednesday however she is going to encourage donal trump to earn the support of republicans and independent voters who have backed her. she is hoping that he picks her as sort of a unity ticket bot given his dominant performente in the primaryes very unlikely',
                      'well end road nikki haley former un ambassador donald trump former governor south carolina dropping race today donald Trump dominated super tuesday course shock question staying reality every state except vermont trump convincingly home state south car Carolina beat 20 points beat 60 40 north carolina beat 73 23 virginia beat 63 34 particularly virginia virginia moderate republican state left wing republican state right wing republicanState beat soundly state california beat 78 18 suggests really shot nomination iowa one shot nomination Iowa pointed rhonda desantis dropped early also niki haley stayed hope win new hampshire overperformed little bit stayed past mainly think put flag rest republican party trump lose november.')
print(scores)


{'rouge1': Score(precision=0.16363636363636364, recall=0.1935483870967742, fmeasure=0.17733990147783252), 'rouge2': Score(precision=0.03669724770642202, recall=0.043478260869565216, fmeasure=0.03980099502487562), 'rougeL': Score(precision=0.16363636363636364, recall=0.1935483870967742, fmeasure=0.17733990147783252)}
